In [10]:
import torch
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

from torch import nn

from sklearn.preprocessing import StandardScaler, MinMaxScaler
standard = StandardScaler()
minmax = MinMaxScaler()

In [11]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Train Set / Validation Set 나누기
- 앞서 train 단계에서 진행한 것과 같이 데이터를 나눠줌

In [13]:
d = pd.read_csv('C:\\Users\\mingu\\Desktop\\data.csv')
d = d.drop('column_a',axis = 1)
d['y'] = d['y']-1
d.loc[(d['y'] != 0) & (d['y'] != 1) , 'y'] = 2
d

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x170,x171,x172,x173,x174,x175,x176,x177,x178,y
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,2
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,0
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,2
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,2
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,15,16,12,5,-1,-18,-37,-47,-48,1
11496,-47,-11,28,77,141,211,246,240,193,136,...,-65,-33,-7,14,27,48,77,117,170,0
11497,14,6,-13,-16,10,26,27,-9,4,14,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,2
11498,-40,-25,-9,-12,-2,12,7,19,22,29,...,121,135,148,143,116,86,68,59,55,2


In [14]:
# label의 비율을 고려하여 train set / validation set 생성
d_0 = d[d['y']==0].copy()
d_0_train = d_0[:1800].copy()
d_0_test = d_0[1800:].copy()

d_1 = d[d['y']==1].copy()
d_1_train = d_1[:1800].copy()
d_1_test = d_1[1800:].copy()

d_2 = d[d['y']==2].copy()
d_2_train = d_2[:5400].copy()
d_2_test = d_2[5400:].copy()

train = pd.concat([d_0_train, d_1_train, d_2_train])
test = pd.concat([d_0_test, d_1_test, d_2_test])

train_x = train.drop(['y'], axis = 1).copy()
train_y = train['y'].copy()

test_x = test.drop(['y'], axis = 1).copy()
test_y = test['y'].copy()

In [15]:
# EEG 신호의 noise를 고려해주기 위해 정규화 진행
from sklearn.preprocessing import StandardScaler
standard = StandardScaler()

train_x_st = standard.fit_transform(train_x)
test_x_st = standard.fit_transform(test_x)

# 모델 가져오기
- 앞서 train 단계에서 만든 모델을 그대로 가져와서 정의

In [16]:
class myRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, out_size):
        
        super().__init__()

        self.features = torch.nn.Sequential(    
            
            #torch.nn.Linear(input_size, hidden_size),
            #torch.nn.ReLU(inplace=True),
            #torch.nn.Dropout(0.2), 
            
            #torch.nn.Linear(hidden_size, 256),
            #torch.nn.Linear(256,256),
            #torch.nn.Linear(256,128),
            #torch.nn.Linear(128, hidden_size),
            #torch.nn.ReLU(),    

            #torch.nn.RNN(input_size=input_size,
            #torch.nn.LSTM(input_size=input_size,
            #torch.nn.GRU(input_size=input_size,
            torch.nn.LSTM(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True,
                          bidirectional=False,
                          dropout=0.3)
    )
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, 20),
            torch.nn.Linear(20,40),
            torch.nn.Linear(40,20),
            torch.nn.Linear(20, hidden_size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.3),
        )
        
        self.output = torch.nn.Linear(hidden_size, out_size)

    def forward(self, x):
        features, _ = self.features(x)
        output = self.classifier(features)
        output = self.output(output)
        return output
    
net = myRNN(15,40,2,3)
net.to(device)

myRNN(
  (features): Sequential(
    (0): LSTM(15, 40, num_layers=2, batch_first=True, dropout=0.3)
  )
  (classifier): Sequential(
    (0): Linear(in_features=40, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=40, bias=True)
    (2): Linear(in_features=40, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=40, bias=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=40, out_features=40, bias=True)
    (6): ReLU(inplace=True)
    (7): Dropout(p=0.3, inplace=False)
  )
  (output): Linear(in_features=40, out_features=3, bias=True)
)

# 파라미터 업로드
- 앞서 train 단계에서 만들어둔 파라미터 파일의 경로로부터 파라미터를 가져옴

In [17]:
# 가져올 경로 설정
PATH_param = "C:\\Users\\mingu\\Desktop\\state_dict_model.pt"

# 로드
net.load_state_dict(torch.load(PATH_param))
net.eval()

myRNN(
  (features): Sequential(
    (0): LSTM(15, 40, num_layers=2, batch_first=True, dropout=0.3)
  )
  (classifier): Sequential(
    (0): Linear(in_features=40, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=40, bias=True)
    (2): Linear(in_features=40, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=40, bias=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=40, out_features=40, bias=True)
    (6): ReLU(inplace=True)
    (7): Dropout(p=0.3, inplace=False)
  )
  (output): Linear(in_features=40, out_features=3, bias=True)
)

In [18]:
# 파라미터 확인
net.state_dict()

OrderedDict([('features.0.weight_ih_l0',
              tensor([[ 0.3825,  0.1696,  0.3888,  ..., -0.0149, -0.2796, -1.0081],
                      [ 1.0355,  0.5075,  0.2560,  ...,  0.2613,  0.3488, -0.2009],
                      [-0.4147, -0.5194, -0.4410,  ..., -0.0834,  0.1688,  0.4171],
                      ...,
                      [-0.6725, -0.4227, -0.0128,  ..., -0.2711, -0.3587, -0.4195],
                      [-0.1928, -0.1214,  0.5168,  ...,  0.1447, -0.2058, -0.0991],
                      [ 0.2795,  0.3319,  0.5523,  ...,  0.0381,  0.1327,  0.0728]])),
             ('features.0.weight_hh_l0',
              tensor([[ 0.5270, -0.2079, -0.4816,  ..., -0.3560,  0.3740,  0.4470],
                      [-0.0881, -0.4017,  0.0637,  ..., -1.4737, -1.3834,  1.5079],
                      [-0.3759, -0.4179, -0.2526,  ..., -0.3713,  0.2377, -1.0464],
                      ...,
                      [-0.5668, -0.5449, -0.3781,  ..., -0.5061,  0.1073, -0.1213],
                     

# Inference

## Sample Data
- Inference를 위해 Validation set에서 각 label에 해당하는 데이터를 가져옴

In [19]:
# label = 0인 Validation set의 sample
test_x.iloc[0] # 원본 데이터에서의 index : 9003
test_x.iloc[495] # 원본 데이터에서의 index : 11487

x1       98
x2      107
x3      121
x4      143
x5      166
       ... 
x174     95
x175    107
x176    111
x177    110
x178    119
Name: 11487, Length: 178, dtype: int64

In [20]:
# label = 1인 Validation set의 sample
test_x.iloc[504] # 원본 데이터에서의 index : 8872
test_x.iloc[996] # 원본 데이터에서의 index : 11484

x1     -18
x2     -12
x3      -5
x4      -4
x5      -3
        ..
x174   -41
x175   -30
x176   -26
x177   -17
x178   -17
Name: 11484, Length: 178, dtype: int64

In [22]:
# label = 2인 Validation set의 sample
test_x.iloc[1003] # 원본 데이터에서의 index : 9055
test_x.iloc[2496] # 원본 데이터에서의 index : 11494

x1     -39
x2     -16
x3     -13
x4     -14
x5     -68
        ..
x174   -66
x175   -49
x176   -34
x177   -31
x178   -52
Name: 11494, Length: 178, dtype: int64

## Inference 진행
- 위에서 언급한 각각의 sample에 대해 inference 진행
- 원본 데이터 d를 전처리하고 정규화한 validation set이므로 원본 데이터의 인덱스가 아닌 validation set의 인덱스를 넣어줘야 함
- label 0 : 0, 495
- label 1 : 504, 996
- label 2 : 1003, 2496
- 랜덤한 input을 위해 randint 함수로 랜덤한 정수를 만들고, 120의 길이만큼 데이터를 가져옴
- 랜덤한 범위에 대해 feature를 잘라 가져오기 때문에 첫 시행에서 잘못 예측하더라도 다음 예측에서는 대부분 옳은 예측을 하는 경우가 많았음
- 파라미터를 업로드하고, 가져온 sample data를 myRNN의 input 형태에 맞게 view함수를 통해 shape 조절
- .eval() 함수를 호출하여 평가단계에서 진행
- softmax 함수와 argmax 함수를 통해 가장 큰 값을 가지는 인덱스를 구해서 result를 출력

In [33]:
a = torch.tensor(test_x_st[1003])
r2 = random.randint(0,40)
a = a[r2:r2+120]

device = "cuda" if torch.cuda.is_available() else "cpu"
net = net.to(device)

model_state_dict = torch.load(PATH_param, map_location=device)
net.load_state_dict(model_state_dict)

net.eval()
X = a.view(1,8,15).to(torch.float32)
with torch.no_grad():
    y_vd_pred = net(X.to(device))
y_vd_pred = y_vd_pred[:,-1,:]            
result = torch.argmax(y_vd_pred.cpu().softmax(1),dim=1) 
print(result)

tensor([2])


# 분석 결과
- 1. EEG 신호가 sequential data이기 때문에, 전반적으로 높은 예측을 보여줄 것이라 생각한 RNN모델을 사용하였다. 수업시간에 다룬 3가지 방법(vanila RNN, GRU, LSTM) 중에서 LSTM 이 가장 좋은 퍼포먼스를 보여주었기 때문에 LSTM을 선택했다.
- 2. 만들고자 하는 모델은 사용자의 EEG sequence를 넣었을때 그 사람이 어떤 상태(간질발작, 종양, 건강)에 놓여있는지를 파악하는 모델이다. 측정을 위한 전극의 위치나 개수에 따라 그 신호의 개수가 달라질 수 있을 것이라 생각하였기 때문에, 임의의 랜덤한 부분에서 120개의 신호를 가져올 수 있도록 data_iter와 모델을 설계하였다. 이는 inference부분에도 동일하게 구현해두었다.
- 3. 학습결과 전반적으로 높은 accuracy를 보여주었기에 vallidation data를 input으로 입력했을 때 결과 역시 대부분의 경우에 제대로 예측했다. 그래도 제대로 파악하지 못하는 몇가지 경우(test_x_st[1003], label = 2)를 분석하기 위해 BMEAI_Project_1_train.ipynb 파일에서 신호의 파형을 출력해 보았다. test_x_st[1003]의 경우에는 원래 label이 2지만, 실제 inference를 돌려보면 1로 예측하는 경우가 존재했다. 이는 아래의 그림에서 보듯이 label이 1인 [996]과 비슷한 패턴이 [1003]의 파형에 존재하기 때문으로 사료된다. 랜덤한 범위에서 L개의 데이터만을 추출하도록 설계해 두었기 때문에, 그 유사한 부분을 함께 가져오면서 잘못된 예측을 할 가능성이 생긴 것 같다. 어쩌면 label 2를 결정하는 부분이 파형에 존재하는 잘은 변화들 때문인 듯 한데, [1003]의 파형은 [2496]과는 다르게 그 자잘함이 눈에 띄지는 않는다. 하지만, inference를 다시 진행하여 새로운 L의 범위를 랜덤하게 받아오면 제대로 예측하는 것을 확인할 수 있었다. 
<p align="center"><img src="https://user-images.githubusercontent.com/65170165/173314960-42de1884-67fd-42c2-bbe1-58e913b7418f.png" width="900" /></p>
- 4. 3에서의 예 뿐만 아니라 서로 다른 label을 가지고 있으나 유사한 부분이 신호에 존재하는 경우에는 예측이 틀리는 경우가 존재하였다. 이는 아무래도 랜덤하게 추출하는 과정에서 신호의 이어짐이 끊긴 이유도 있을 테지만, 기장 큰 이유는 사람이 신호의 파형만을 보고 EEG 신호의 노이즈를 판별하여 분석해내기에는 어려움이 따르기 때문이라고 생각한다. 반면 같은 이유로 inference를 할때 랜덤한 구간 L을 설정해주기 때문에 잘못 예측한 경우 역시 inference를 다시 진행해보면 대다수가 원래의 label과 일치하는 것을 확인할 수 있었다.
- 5. 무의식적인 뒤척거림이나 눈 움직임에 의해 신호에 큰 변화가 일어나는 EEG 신호를 사용했기 때문에 이를 어느정도 보정해주기 위한 적절한 표준화 방법이 필요했다. 따라서 신호의 noise를 제거해주기 위해 moving average와 MinMaxScaler, StandardScaler를 각각 적용해보았는데 StandardScaler의 경우에 가장 좋은 결과를 나타냈기 때문에 이 method를 선택해서 학습을 진행했다. 
- 6. 모델의 일반성을 확인해보기 위해서 다른 EEG data(945 rows, 1149 columns)를 적용해서 classification을 진행하였으나, 이 데이터에 대해서는 좋은 결과를 얻지 못했다. 우선적으로 데이터의 7개의 label들에 대한 분포가 고르지 않았다. 또한 본 프로젝트에 사용한 데이터(11500 rows, 179 columns)에 비해 column의 수는 많지만, 그 수많은 EEG 신호에 대한 노이즈를 잡아 결과를 얻기에는 object의 수가 너무 적었기 때문이라고 생각한다.
- 7. 결론적으로 EEG 신호처럼 노이즈가 많은 데이터를 분석하기 위해서는 이를 잘 설명해줄 수 있을 만한 충분한 수의 데이터와 적절한 scaler 방법이 필요하다고 생각한다. 또한 앞서 말했듯이 사람이 이러한 노이즈를 잡아내고 패턴을 분석해서 그에 해당하는 인사이트를 도출하기는 너무 어렵기 때문에 EEG 신호와 같은 의학적 분야에 인공지능이 꼭 필요할 것이라고 결론을 내릴 수 있었다.

# 배운 점
- 데이터를 선정하고, 전처리하기까지의 어려움은 크게 없었던 것 같다. 하지만 이 데이터를 우리가 원하는 형태로 바꿔주고, 특히 모델과의 shape를 맞추는 데 시간이 굉장히 많이 걸렸던 것 같다. 이렇게 모델의 shape를 맞춰서 학습을 시작하는 것까지는 성공을 했지만, 그 뒤에 이어진 학습 단계에서 accuracy가 local optimum 에 빠지는 상황이 계속 발생해서 바꿀 수 있는 건 모두 바꿔봤던 것 같다. learning rate, epoch, optimization, loss function 그리고 각종 사이즈까지 모두 바꿔봤지만 도저히 상황이 나아지지 않았고, 밑져야 본전이라는 마음가짐으로 그냥 LSTM 함수의 num_layer를 바꾸었더니 이렇게 좋은 결과를 만들 수 있었다. 인터넷을 찾아보며 사서 고생을 정말 많이 한 것 같은데, 오히려 간단한 매개변수를 하나 바꿔줌으로써 완전히 상반된 결과를 만들 수 있다는  것이 신기한 한편 허무하기도 했다. 하지만 이런 과정을 통해 수업시간에 배운 optimize algorithm 외에도 수많은 종류가 있다는 것도 알 수 있었고, 그 중에 하나인 Adam을 선택해서 더 나은 accuracy를 얻을 수 있었다. 또한 수업시간에 수업을 들으면서도 이렇게 학습한 모델을 실제로 어떻게 사용하는지는 항상 궁금했는데, Inference 과정을 구현해보면서 그 방법을 배운 것 같아 재밌었다. 수업시간에 배우는 이론도, 그리고 실습코드를 사용해서 과제를 하는 것 역시 중요하지만 그래도 제일 기억에 남는 것은 역시 직접 시간을 들여서 찾아보고, 결론을 도출하기까지의 과정인 것 같다. 결과적으로 우리는 바이오메디컬인공지능 프로젝트를 통해 직접 데이터를 불러오고 모델에 적용하면서 그 shape 와 각종 하이퍼파라미터들을 맞추는 인공지능의 핵심을 배웠고, 그보다 중요한 하나의 인공지능을 구현해보았다는 경험을 얻을 수 있었다.

# 질문에 대한 답변
 - Q. class간 비율이 1:1:3으로 normal이 상대적으로 많은데 이로인한 문제가 없을까요?
 - Q. class별로 accuracy를 각각 분석하여 주세요.

 - 위 질문들에 대한 답을 위해 classification report와 confusion matrix를 출력했다. 그 결과는 아래와 같았다. 
<p align="center"><img src="https://user-images.githubusercontent.com/65170165/174334834-cf138f57-f5af-4dd1-a266-74724441eae2.png" width="900" /></p>

## Q. class간 비율이 1:1:3으로 normal이 상대적으로 많은데 이로 인한 문제가 없을까요?
 - 사실 첫번째 질문과 두번째 질문에 대한 답을 이 두개의 결과로부터 모두 알아볼 수 있을 것 같다. 에포크를 많이 돌려서 실험한 결과 accuracy는 계속 꾸준히, 그리고 굉장히 높은 수치까지 성장한 것을 확인했지만, 아래의 report에서 볼 수 있듯이 전체적인 accuracy는 생각보다 낮게 측정되었다. normal이 상대적으로 많이 데이터였기 때문에 데이터가 어느 정도 불균형하다는 것을 염두에 두고, accuracy보다는 f1-score에 좀 더 비중을 두고 분석하는 것이 좋을 수 있다고 생각하였다. 우리가 우선적으로 찾아내고 싶었던 target인 Epileptic_Seizures (label=0)에 대해서는 그 score가 0.97로 굉장히 높은 것을 확인할 수 있었고, 이에 대해서는 좋은 성능을 보이는 모델이라 할 수 있을 듯 하다. 또한 confusion matrix 에도 나타나 있듯이, 총 500개의 label 0 에 대해서 483개의 좋은 예측을 한 모델을 만들 수 있었다. 하지만 상대적으로 Tumor에 대한 f1-score 는 상당히 낮은 것을 확인할 수 있었다. 이는 아무래도 전체적인 신호의 파형이 tumor와 normal의 경우가 비슷한 부분들이 존재하기 때문에 tumor를 정확히 파악하기에는 tumor label에 대한 비율이 살짝 부족하지 않았을까 싶다. 결론적으로 상대적인 비율이 우리가 주 목적으로 한 Epileptic_Seizures를 알아내는 데에는 지장이 없었지만, 다른 label, 특히 Tumor를 파악하는 데에는 영향을 미쳤다고 생각한다.  

 ## Q. class별로 accuracy를 각각 분석하여 주세요.
 
 class | accuracy
--- | ---
Epileptic_Seizures (0) | 0.966
Tumor (1) | 0.652
Normal (2) | 0.871